In [1]:
%run common.ipynb

In [2]:
# Parameters
# Chose variables to create the model
input_cols = ['shape', 'profile_entrance', 'profile_exit', 'rising_entrance', 'rising_exit', 'shards', 'feathering', 'entrance_mounding', 'center_mounding', 'exit_mounding']
result_col = 'result'
# Load the train and the validation datasets for blade class prediction
train_dataset_file = 'DATA/Train/blade_prediction_train.xlsx'
test_dataset_file = 'DATA/Validation/blade_prediction_validation.xlsx'

In [3]:
# The data are displayed on a table
display_input_data(
    train_dataset_file,
    input_cols
)

,shape_E,shape_indeterminate,profile_entrance_V,profile_exit_V,rising_entrance_bilateral,rising_entrance_single,rising_exit_bilateral,rising_exit_single,shards_present,feathering_present,entrance_mounding_bilateral,entrance_mounding_single,center_mounding_bilateral,center_mounding_single,exit_mounding_bilateral,exit_mounding_single
0,False,True,True,True,False,True,False,True,False,False,False,True,False,False,False,True
1,False,True,False,True,False,True,True,False,False,False,False,True,False,False,False,True
2,False,True,False,True,False,True,True,False,False,False,False,True,False,False,True,False
3,False,True,False,True,False,True,True,False,False,False,False,True,False,False,True,False
4,False,True,False,True,False,True,True,False,False,False,False,True,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,True,False,True,True,False,True,False,True,False,False,False,True,False,False,False,True
276,True,False,True,False,True,False,True,False,False,False,False,False,False,False,False,False
277,True,False,True,True,False,False,False,True,False,False,False,False,False,False,False,False
278,True,False,True,True,False,False,False,False,False,True,False,False,False,False,False,False


In [4]:
# The two datasets are tested for compatibility 
check_input_data_indices(train_dataset_file, test_dataset_file, input_cols)

Datasets are compatible


In [5]:
# Logistic regression model
model = train_model(
    train_dataset_file,
    input_cols,
    result_col
)
model

LogisticRegression(random_state=0, solver='liblinear')

In [6]:
# Test for overfitting
# Evaluation of the training
x_train, y_train = prepare_dataset(train_dataset_file, input_cols, result_col)
model.score(x_train, y_train)

0.8321428571428572

In [7]:
# 10 variables were used 
evaluate_model(
    model,
    test_dataset_file,
    input_cols,
    result_col
)

0.7611940298507462

In [ ]:
# Model optimisation, search for the optimal parameter combination
# Printing score best and worst model
run_input_optimizer(train_dataset_file, test_dataset_file, input_cols, result_col)

Testing combinations of length 2
Testing combinations of length 3
